### GPU CHECKS AND IMPORTING DATA

All work was done in AWS, so google colab related code is removed as well as downloading of N-HPatches etc.

In [0]:
%cd keras_triplet_descriptor/

!pip install humanize
!pip install gputil
!pip install psutil
!pip install talos
!pip install sklearn
!pip install opencv-python
!pip install joblib
!pip install dill
!pip install tabulate
import psutil
import humanize
import os
import GPUtil as GPU

import sys
import json
import os
import glob
import time
import tensorflow as tf
import numpy as np
import cv2
import random
import talos
import pandas as pd
import matplotlib.pyplot as plt

import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, Reshape
from keras.layers import Conv2D, Conv3D, MaxPooling2D, BatchNormalization 
from keras.layers import Input, UpSampling2D, concatenate  
from keras.activations import sigmoid, relu, elu
from talos import live, Reporting
from talos.model.normalizers import lr_normalizer
from keras.losses import mean_squared_error, binary_crossentropy
from keras.optimizers import Adam, Nadam, SGD, RMSprop, Adagrad, Adadelta
from sklearn.preprocessing import MinMaxScaler
from keras.models import load_model
 
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)   

#!wget -O hpatches_data.zip https://imperialcollegelondon.box.com/shared/static/ah40eq7cxpwq4a6l4f62efzdyt8rm3ha.zip
  
#!unzip -q ./hpatches_data.zip
#!rm ./hpatches_data.zip

hpatches_dir = './hpatches'
splits_path = './splits.json'

splits_json = json.load(open(splits_path, 'rb'))
split = splits_json['a']

train_fnames = split['train']
test_fnames = split['test']

seqs = glob.glob(hpatches_dir+'/*')
seqs = [os.path.abspath(p) for p in seqs]   
seqs_train = list(filter(lambda x: x.split('/')[-1] in train_fnames, seqs)) 
seqs_test = list(filter(lambda x: x.split('/')[-1] in split['test'], seqs))

from read_data import HPatches, DataGeneratorDesc, hpatches_sequence_folder, DenoiseHPatches, tps
from utils import generate_desc_csv, plot_denoise, plot_triplet

### EXPERIMENTS AND TRAINING THE FINAL DESCRIPTOR

Some hyperparameter optimisation was done here but not mentioned in the report. This is due to the fact that I ran out of space in the report (max length 4 pages) and found a very good set of hyperparameters in a research paper which implemeted a similar model. 

https://arxiv.org/pdf/1705.10872.pdf

The final version of the model is trained at the end of this notebook.

In [0]:
denoise_model = load_model('denoise.h5')

In [0]:
hPatches = HPatches(train_fnames=train_fnames, test_fnames=test_fnames, denoise_model=denoise_model, use_clean=False)
training_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=1), num_triplets=1000000, batch_size = 512)
val_generator = DataGeneratorDesc(*hPatches.read_image_file(hpatches_dir, train=0), num_triplets=100000, batch_size = 512)

Using denoised patches
100%|██████████| 116/116 [00:35<00:00,  2.52it/s]
Denoising patches...
100%|██████████| 15589/15589 [02:40<00:00, 97.45it/s]


100%|██████████| 400000/400000 [00:04<00:00, 87416.89it/s]

Using denoised patches


100%|██████████| 116/116 [00:21<00:00,  5.47it/s]
Denoising patches...
100%|██████████| 9525/9525 [01:37<00:00, 98.00it/s]


100%|██████████| 50000/50000 [00:00<00:00, 88593.44it/s]


In [0]:
xTrain, yTrain = training_generator.__getitem__(0)
xVal, yVal = val_generator.__getitem__(0)

In [0]:
def talosDescriptorOptimisation(xTrain, yTrain, xVal, yVal, params):
  
  init_weights = keras.initializers.he_normal()
  
  descriptor_model = Sequential()
  
  descriptor_model.add(Conv2D(32, 3, padding='same', activation = 'relu', input_shape = (32, 32, 1) , kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(32, 3, padding='same', activation = 'relu', kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(64, 3, padding='same', activation = 'relu', strides=2, kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(64, 3, padding='same', activation = 'relu', kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(128, 3, padding='same', activation = 'relu', strides=2, kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(128, 3, padding='same', activation = 'relu', kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))
  descriptor_model.add(Dropout(rate = params['dropout']))
  descriptor_model.add(Conv2D(128, 8, padding='valid', kernel_initializer=init_weights))

  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))  
    
  descriptor_model.add(Reshape((128,)))
  
  def triplet_loss(x):

    output_dim = 128
    a, p, n = x
    _alpha = 1.0
    L1L2 = params['L1L2']
    
    if(L1L2 == 'L1'):
      positive_distance = K.mean(K.abs(a - p), axis = -1)
      negative_distance = K.mean(K.abs(a - n), axis = -1)
    else:
      positive_distance = K.mean(K.square(a - p), axis = -1)
      negative_distance = K.mean(K.square(a - n), axis = -1)
  
    return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)

  xa = Input(shape=(32, 32, 1), name='a')
  xp = Input(shape=(32, 32, 1), name='p')
  xn = Input(shape=(32, 32, 1), name='n')
  ea = descriptor_model(xa)
  ep = descriptor_model(xp)
  en = descriptor_model(xn)

  loss = Lambda(triplet_loss)([ea, ep, en])
    
  descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)
  descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=params['optimizer'](lr=lr_normalizer(params['lr'],params['optimizer'])))

  out = descriptor_model_trip.fit(x = xTrain, y = yTrain,
                   epochs = params['epochs'],
                   verbose=0,
                   validation_data = [xVal, yVal],
                   batch_size = 10)
    
  return out, descriptor_model

In [0]:
param = {
    'L1L2' : ['L1', 'L2'],
    'optimizer' : [Adam, Adagrad, RMSprop, Adadelta],
    'epochs' : [10],
    'lr' : (0.00001, 1, 50),
    'dropout' : (0, 0.90, 15)}

r = talos.Scan(x = xTrain, y = yTrain,
           params = param,
           model = talosDescriptorOptimisation,
           grid_downsample = 0.05,
           dataset_name = 'NHPatches_Descriptor',
           experiment_no = '1',
           x_val = xVal, y_val = yVal,
           disable_progress_bar = False)




  0%|          | 0/300 [00:00<?, ?it/s]


  0%|          | 1/300 [00:08<43:30,  8.73s/it]


  1%|          | 2/300 [00:17<43:01,  8.66s/it]


  1%|          | 3/300 [00:25<42:10,  8.52s/it]


  1%|▏         | 4/300 [00:33<41:28,  8.41s/it]


  2%|▏         | 5/300 [00:42<41:47,  8.50s/it]


  2%|▏         | 6/300 [00:50<40:55,  8.35s/it]


  2%|▏         | 7/300 [00:57<39:08,  8.01s/it]


  3%|▎         | 8/300 [01:06<40:17,  8.28s/it]


  3%|▎         | 9/300 [01:14<40:07,  8.27s/it]


  3%|▎         | 10/300 [01:21<38:31,  7.97s/it]


  4%|▎         | 11/300 [01:30<38:33,  8.01s/it]


  4%|▍         | 12/300 [01:38<38:29,  8.02s/it]


  4%|▍         | 13/300 [01:46<38:14,  8.00s/it]


  5%|▍         | 14/300 [01:54<39:25,  8.27s/it]


  5%|▌         | 15/300 [02:02<38:59,  8.21s/it]


  5%|▌         | 16/300 [02:11<39:49,  8.41s/it]


  6%|▌         | 17/300 [02:20<39:51,  8.45s/it]


  6%|▌         | 18/300 [02:28<38:58,  8.29s/it]


  6%|▋         | 19/300 [02:37<39:42,  8.48s/i

 90%|█████████ | 270/300 [37:38<04:09,  8.31s/it]


 90%|█████████ | 271/300 [37:47<04:08,  8.56s/it]


 91%|█████████ | 272/300 [37:55<03:53,  8.35s/it]


 91%|█████████ | 273/300 [38:03<03:45,  8.34s/it]


 91%|█████████▏| 274/300 [38:12<03:42,  8.55s/it]


 92%|█████████▏| 275/300 [38:20<03:24,  8.17s/it]


 92%|█████████▏| 276/300 [38:29<03:22,  8.43s/it]


 92%|█████████▏| 277/300 [38:38<03:19,  8.66s/it]


 93%|█████████▎| 278/300 [38:46<03:06,  8.49s/it]


 93%|█████████▎| 279/300 [38:55<03:03,  8.73s/it]


 93%|█████████▎| 280/300 [39:03<02:51,  8.57s/it]


 94%|█████████▎| 281/300 [39:11<02:35,  8.18s/it]


 94%|█████████▍| 282/300 [39:20<02:32,  8.45s/it]


 94%|█████████▍| 283/300 [39:29<02:27,  8.66s/it]


 95%|█████████▍| 284/300 [39:36<02:11,  8.24s/it]


 95%|█████████▌| 285/300 [39:45<02:07,  8.50s/it]


 95%|█████████▌| 286/300 [39:55<02:01,  8.71s/it]


 96%|█████████▌| 287/300 [40:02<01:47,  8.30s/it]


 96%|█████████▌| 288/300 [40:10<01:40,  8.38s/it]


 96%|███████

In [0]:
Exp1Data = r.data
Exp1Data = Exp1Data.drop(Exp1Data[Exp1Data.val_loss > '0.5'].index)
Exp1Data = Exp1Data.drop(Exp1Data[Exp1Data.val_loss <= '0.0'].index)
Exp1Data = Exp1Data.dropna()
Exp1Data = Exp1Data.drop(columns=['round_epochs', 'epochs'])
Exp1Data

loss             val_loss                    lr  \
12   0.032189302146434784  0.45342991650104525             0.6600034   
21      0.819526982307434  0.45327138900756836                 1e-05   
24    0.20685001313686371  0.49043130278587344   0.06000940000000001   
33   0.022059301473200323  0.41863422393798827   0.26000740000000006   
37    0.35358330309391023   0.4810577392578125             0.6400036   
44     0.3858364462852478   0.4871972739696503  0.040009600000000006   
54    0.24421680197119713  0.48322041630744933             0.0800092   
56   0.016434440854936838    0.458419543504715   0.36000640000000006   
60    0.03630866259336472  0.43801897168159487             0.7600024   
61    0.14716588109731674   0.4980979084968567   0.20000800000000002   
64    0.02824580082669854  0.45459653735160827             0.5400046   
110   0.03938387557864189   0.4230538785457611             0.8400016   
134   0.06637137979269028   0.4634739577770233             0.6600034   
138   0.09077822864055633   0.4931167632341385              0.600004   
181  0.015876579284667968   0.4535797297954559             0.1400086   
192   0.02478706374531612  0.45619123578071596   0.40000600000000003   
206  0.024249628745019437   0.4830226182937622    0.4800052000000001   
209   0.14661443158984183  0.49139901995658875             0.6200038   
227  0.049455966986715796  0.41503362357616425             0.6800032   
230  0.057072941400110724  0.34321888387203214              0.900001   
232   0.23528686612844468   0.4174801623914391             0.6800032   
235   0.01670846724882722  0.49769162237644193   0.26000740000000006   
236    0.0331964248791337    0.480774387717247             0.6800032   
253    0.2308540776371956   0.4907564640045166              0.800002   
256   0.09544289074838161   0.4861449122428894             0.7600024   
273   0.34785295724868776  0.42690956890583037  0.040009600000000006   
283    0.1136306345462799  0.49697172939777373   0.24000760000000004   
286   0.03614821843802929  0.36246668100357055   0.44000560000000005   
288  0.024814629182219505   0.4664437294006348             0.2800072   

                  dropout                            optimizer L1L2  
12                    0.0   <class 'keras.optimizers.RMSprop'>   L2  
21                    0.0  <class 'keras.optimizers.Adadelta'>   L1  
24    0.12000000000000001   <class 'keras.optimizers.Adagrad'>   L1  
33                    0.0   <class 'keras.optimizers.RMSprop'>   L2  
37     0.8400000000000001   <class 'keras.optimizers.Adagrad'>   L1  
44    0.12000000000000001   <class 'keras.optimizers.RMSprop'>   L1  
54   0.060000000000000005      <class 'keras.optimizers.Adam'>   L1  
56   0.060000000000000005   <class 'keras.optimizers.RMSprop'>   L2  
60                   0.54   <class 'keras.optimizers.RMSprop'>   L2  
61   0.060000000000000005  <class 'keras.optimizers.Adadelta'>   L1  
64    0.24000000000000002   <class 'keras.optimizers.RMSprop'>   L2  
110   0.24000000000000002   <class 'keras.optimizers.RMSprop'>   L2  
134                   0.0   <class 'keras.optimizers.Adagrad'>   L2  
138                   0.0   <class 'keras.optimizers.Adagrad'>   L1  
181                   0.0   <class 'keras.optimizers.Adagrad'>   L2  
192   0.12000000000000001   <class 'keras.optimizers.RMSprop'>   L2  
206   0.18000000000000002   <class 'keras.optimizers.RMSprop'>   L2  
209   0.12000000000000001   <class 'keras.optimizers.Adagrad'>   L1  
227   0.42000000000000004   <class 'keras.optimizers.RMSprop'>   L2  
230                   0.0   <class 'keras.optimizers.Adagrad'>   L2  
232   0.18000000000000002   <class 'keras.optimizers.Adagrad'>   L1  
235   0.12000000000000001   <class 'keras.optimizers.Adagrad'>   L2  
236   0.36000000000000004      <class 'keras.optimizers.Adam'>   L2  
253   0.48000000000000004   <class 'keras.optimizers.Adagrad'>   L1  
256                   0.0   <class 'keras.optimizers.Adagrad'>   L2  
273                   0.0  <class 'keras

In [0]:
param = {
    'L1L2' : ['L2'],
    'optimizer' : [Adagrad, RMSprop],
    'epochs' : [10],
    'lr' : (0.00001, 1, 1000),
    'dropout' : [0.3]}

r = talos.Scan(x = xTrain, y = yTrain,
           params = param,
           model = talosDescriptorOptimisation,
           grid_downsample = 0.1,
           dataset_name = 'NHPatches_Descriptor',
           experiment_no = '2',
           x_val = xVal, y_val = yVal,
           disable_progress_bar = False)




  0%|          | 0/200 [00:00<?, ?it/s]


  0%|          | 1/200 [00:07<24:31,  7.39s/it]


  1%|          | 2/200 [00:16<26:25,  8.01s/it]


  2%|▏         | 3/200 [00:24<26:18,  8.01s/it]


  2%|▏         | 4/200 [00:33<26:27,  8.10s/it]


  2%|▎         | 5/200 [00:41<26:17,  8.09s/it]


  3%|▎         | 6/200 [00:49<26:14,  8.12s/it]


  4%|▎         | 7/200 [00:57<26:09,  8.13s/it]


  4%|▍         | 8/200 [01:05<25:55,  8.10s/it]


  4%|▍         | 9/200 [01:13<25:47,  8.10s/it]


  5%|▌         | 10/200 [01:21<25:34,  8.08s/it]


  6%|▌         | 11/200 [01:30<25:49,  8.20s/it]


  6%|▌         | 12/200 [01:38<25:41,  8.20s/it]


  6%|▋         | 13/200 [01:46<25:40,  8.24s/it]


  7%|▋         | 14/200 [01:55<25:33,  8.25s/it]


  8%|▊         | 15/200 [02:03<25:23,  8.23s/it]


  8%|▊         | 16/200 [02:11<25:15,  8.23s/it]


  8%|▊         | 17/200 [02:19<24:57,  8.18s/it]


  9%|▉         | 18/200 [02:27<24:43,  8.15s/it]


 10%|▉         | 19/200 [02:35<24:43,  8.20s/i

In [0]:
Exp1Data = r.data
Exp1Data = Exp1Data.drop(Exp1Data[Exp1Data.val_loss > '0.45'].index)
Exp1Data = Exp1Data.drop(Exp1Data[Exp1Data.val_loss <= '0.0'].index)
Exp1Data = Exp1Data.dropna()
Exp1Data = Exp1Data.drop(columns=['round_epochs', 'epochs'])
Exp1Data

loss             val_loss                   lr dropout  \
6    0.029123722482472657   0.4119798630475998           0.54100459     0.3   
12   0.024080299958586692   0.4343499630689621           0.51600484     0.3   
45   0.042338790744543074   0.4038113743066788           0.90500095     0.3   
49   0.008962098136544227  0.44893929064273835           0.43500565     0.3   
59    0.03962194137275219   0.4290470451116562  0.34900651000000005     0.3   
67   0.034034882485866544  0.44685842990875246  0.17400826000000003     0.3   
74    0.04365070946514606  0.44920477271080017  0.22500775000000003     0.3   
88    0.19454144984483718   0.4395829439163208           0.02100979     0.3   
90    0.03548243148252368   0.3909574419260025  0.46700533000000005     0.3   
108  0.033804242871701716   0.4105414509773254           0.94400056     0.3   
118  0.057899995148181914   0.3800204277038574            0.5100049     0.3   
133   0.03613777104765177   0.4286235928535461  0.46800532000000006     0.3   
134   0.10157632678747178   0.4227862924337387           0.92500075     0.3   
163  0.016230018297210335   0.4363508075475693           0.31600684     0.3   
168   0.03448911076411605   0.3807806774973869   0.4890051100000001     0.3   
181  0.036928610131144526   0.4125595197081566           0.54800452     0.3   
198  0.030396737158298492   0.4494719713926315  0.10800892000000001     0.3   

                              optimizer L1L2  
6    <class 'keras.optimizers.RMSprop'>   L2  
12   <class 'keras.optimizers.Adagrad'>   L2  
45   <class 'keras.optimizers.Adagrad'>   L2  
49   <class 'keras.optimizers.RMSprop'>   L2  
59   <class 'keras.optimizers.RMSprop'>   L2  
67   <class 'keras.optimizers.Adagrad'>   L2  
74   <class 'keras.optimizers.Adagrad'>   L2  
88   <class 'keras.optimizers.Adagrad'>   L2  
90   <class 'keras.optimizers.RMSprop'>   L2  
108  <class 'keras.optimizers.RMSprop'>   L2  
118  <class 'keras.optimizers.Adagrad'>   L2  
133  <class 'keras.optimizers.Adagrad'>   L2  
134  <class 'keras.optimizers.Adagrad'>   L2  
163  <class 'keras.optimizers.RMSprop'>   L2  
168  <class 'keras.optimizers.RMSprop'>   L2  
181  <class 'keras.optimizers.Adagrad'>   L2  
198  <class 'keras.optimizers.Adagrad'>   L2

In [0]:
param = {
    'L1L2' : ['L2'],
    'optimizer' : [RMSprop],
    'epochs' : [25],
    'lr' : (0.00001, 10, 100),
    'dropout' : [0.3]}

r = talos.Scan(x = xTrain, y = yTrain,
           params = param,
           model = talosDescriptorOptimisation,
           dataset_name = 'NHPatches_Descriptor',
           experiment_no = '3',
           x_val = xVal, y_val = yVal,
           disable_progress_bar = False)




  0%|          | 0/100 [00:00<?, ?it/s]


  1%|          | 1/100 [00:09<16:21,  9.91s/it]


  2%|▏         | 2/100 [00:19<15:59,  9.79s/it]


  3%|▎         | 3/100 [00:29<15:52,  9.82s/it]


  4%|▍         | 4/100 [00:38<15:37,  9.76s/it]


  5%|▌         | 5/100 [00:48<15:27,  9.77s/it]


  6%|▌         | 6/100 [00:58<15:14,  9.73s/it]


  7%|▋         | 7/100 [01:08<15:04,  9.72s/it]


  8%|▊         | 8/100 [01:17<15:00,  9.78s/it]


  9%|▉         | 9/100 [01:27<14:45,  9.73s/it]


 10%|█         | 10/100 [01:37<14:29,  9.66s/it]


 11%|█         | 11/100 [01:47<14:26,  9.73s/it]


 12%|█▏        | 12/100 [01:56<14:15,  9.72s/it]


 13%|█▎        | 13/100 [02:06<14:10,  9.77s/it]


 14%|█▍        | 14/100 [02:16<13:57,  9.74s/it]


 15%|█▌        | 15/100 [02:25<13:45,  9.71s/it]


 16%|█▌        | 16/100 [02:35<13:37,  9.73s/it]


 17%|█▋        | 17/100 [02:45<13:24,  9.70s/it]


 18%|█▊        | 18/100 [02:55<13:17,  9.72s/it]


 19%|█▉        | 19/100 [03:04<13:04,  9.69s/i

In [0]:
Exp1Data = r.data
Exp1Data = Exp1Data.drop(Exp1Data[Exp1Data.val_loss > '0.45'].index)
Exp1Data = Exp1Data.drop(Exp1Data[Exp1Data.val_loss <= '0.0'].index)
Exp1Data = Exp1Data.dropna()
Exp1Data = Exp1Data.drop(columns=['round_epochs', 'epochs'])
Exp1Data

loss             val_loss                   lr dropout  \
1    0.012740025762468576  0.27904887348413465            1.3000087     0.3   
11   0.054307717084884646   0.3357808470726013            2.5000075     0.3   
18    0.05221538711339235  0.44815750122070314   3.8000062000000003     0.3   
19   0.009793102741241455   0.3508525609970093            0.9000091     0.3   
20                    0.0   0.3187064900994301            1.4000086     0.3   
31                    0.0   0.4032825991511345            0.3000097     0.3   
33   0.018462853133678438  0.31792865246534346   1.8000082000000002     0.3   
35                    0.0   0.4196945488452911  0.20000980000000002     0.3   
38   0.012421085452660918  0.43107436001300814            2.4000076     0.3   
47    0.08274224996566773   0.3792426323518157            3.4000066     0.3   
48   0.015960846096277237  0.43692160248756406            1.5000085     0.3   
49   0.012300381809473038   0.3288886405527592            1.6000084     0.3   
67  0.0006932186894118786   0.4477788686752319            0.4000096     0.3   
75   0.018350712954998016  0.37550124526023865            3.3000067     0.3   
81    0.03703721687197685  0.36716077029705046   3.1000069000000003     0.3   
82   0.002997894212603569  0.35579861998558043            0.8000092     0.3   
85   0.011387495696544648  0.38147366493940355            2.3000077     0.3   
90   0.014908884768374264  0.35024564489722254            0.7000093     0.3   
92    0.03212754279375076  0.34701505303382874   1.9000081000000002     0.3   
93    0.02385903298854828  0.18799381591379644   1.7000083000000001     0.3   

                             optimizer L1L2  
1   <class 'keras.optimizers.RMSprop'>   L2  
11  <class 'keras.optimizers.RMSprop'>   L2  
18  <class 'keras.optimizers.RMSprop'>   L2  
19  <class 'keras.optimizers.RMSprop'>   L2  
20  <class 'keras.optimizers.RMSprop'>   L2  
31  <class 'keras.optimizers.RMSprop'>   L2  
33  <class 'keras.optimizers.RMSprop'>   L2  
35  <class 'keras.optimizers.RMSprop'>   L2  
38  <class 'keras.optimizers.RMSprop'>   L2  
47  <class 'keras.optimizers.RMSprop'>   L2  
48  <class 'keras.optimizers.RMSprop'>   L2  
49  <class 'keras.optimizers.RMSprop'>   L2  
67  <class 'keras.optimizers.RMSprop'>   L2  
75  <class 'keras.optimizers.RMSprop'>   L2  
81  <class 'keras.optimizers.RMSprop'>   L2  
82  <class 'keras.optimizers.RMSprop'>   L2  
85  <class 'keras.optimizers.RMSprop'>   L2  
90  <class 'keras.optimizers.RMSprop'>   L2  
92  <class 'keras.optimizers.RMSprop'>   L2  
93  <class 'keras.optimizers.RMSprop'>   L2

Training and evaluation of the final descriptor model as well as well as evaluation of the entire system.

In [0]:
def get_descriptor_model(shape):
  
  init_weights = keras.initializers.he_normal()
  
  descriptor_model = Sequential()
  
  descriptor_model.add(Conv2D(32, 3, padding='same', activation = 'relu', input_shape = shape , kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(32, 3, padding='same', activation = 'relu', kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(64, 3, padding='same', activation = 'relu', strides=2, kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(64, 3, padding='same', activation = 'relu', kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(128, 3, padding='same', activation = 'relu', strides=2, kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))

  descriptor_model.add(Conv2D(128, 3, padding='same', activation = 'relu', kernel_initializer=init_weights))
  descriptor_model.add(Activation('relu'))
  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))
  descriptor_model.add(Dropout(0.3))
  descriptor_model.add(Conv2D(128, 8, padding='valid', kernel_initializer=init_weights))

  descriptor_model.add(BatchNormalization(axis = -1, gamma_initializer = 'ones', beta_initializer = 'zeros', trainable = True))  
    
  descriptor_model.add(Reshape((128,)))

  return descriptor_model

def triplet_loss(x):

  output_dim = 128
  a, p, n = x
  _alpha = 1.0
  positive_distance = K.mean(K.square(a - p), axis = -1)
  negative_distance = K.mean(K.square(a - n), axis = -1)
  
  return K.expand_dims(K.maximum(0.0, positive_distance - negative_distance + _alpha), axis = 1)


shape = (32, 32, 1)
xa = Input(shape=shape, name='a')
xp = Input(shape=shape, name='p')
xn = Input(shape=shape, name='n')
descriptor_model = get_descriptor_model(shape)
ea = descriptor_model(xa)
ep = descriptor_model(xp)
en = descriptor_model(xn)

loss = Lambda(triplet_loss)([ea, ep, en])

descriptor_model_trip = Model(inputs=[xa, xp, xn], outputs=loss)

sgd = keras.optimizers.SGD(lr=0.1)
descriptor_model_trip.compile(loss='mean_absolute_error', optimizer=RMSprop(lr = 0.4))

In [0]:
epochs = 50

#es = EarlyStopping(monitor='val_loss', mode='min', baseline=0.085)

descriptor_history_baseline_trip = descriptor_model_trip.fit_generator(generator=training_generator, epochs=50, verbose=1, validation_data=val_generator, use_multiprocessing = True)

generate_desc_csv(descriptor_model, seqs_test, denoise_model = denoise_model, use_clean = False)

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=out/ --task=verification --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=verification

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=out/ --task=matching --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=matching

!python ./hpatches-benchmark/hpatches_eval.py --descr-name=custom --descr-dir=out/ --task=retrieval --delimiter=";"
!python ./hpatches-benchmark/hpatches_results.py --descr=custom --results-dir=./hpatches-benchmark/results/ --task=retrieval

Epoch 1/50
780/781 [============================>.] - ETA: 0s - loss: 0.1664

 36%|███▋      | 145242/400000 [00:02<00:04, 62404.02it/s]

781/781 [==============================] - 74s 95ms/step - loss: 0.1663 - val_loss: 0.2594


 38%|███▊      | 151760/400000 [00:02<00:05, 46651.40it/s]

Epoch 2/50


...

780/781 [============================>.] - ETA: 0s - loss: 0.0531

 23%|██▎       | 91903/400000 [00:02<00:12, 24785.36it/s]

781/781 [==============================] - 77s 99ms/step - loss: 0.0531 - val_loss: 0.0851


100%|██████████| 40/40 [03:45<00:00,  4.56s/it]



>> Running HPatch evaluation for custom
>> Please wait, loading the descriptor files...
>> Descriptor files loaded.
>> Evaluating verification task
Processing verification task 3/3 : 100%|██████████| 1000000/1000000 [01:23<00:00, 11943.17it/s]
>> Verification task finished in 256 secs  
Verification task results:
Mean Average Precision is 0.870655



>> Running HPatch evaluation for custom
>> Please wait, loading the descriptor files...
>> Descriptor files loaded.
>> Evaluating matching task
100%|██████████| 40/40 [01:48<00:00,  3.67s/it]
>> Matching task finished in 109 secs  
Matching task results:
Mean Average Precision is 0.309993





>> Running HPatch evaluation for custom
>> Please wait, loading the descriptor files...
>> Descriptor files loaded.
>> Evaluating retrieval task
>> Please wait, computing distance matrix...
>> Distance matrix done.
Processing retrieval task: 100%|██████████| 10000/10000 [03:33<00:00, 46.67it/s]
>> Retrieval task finished in 229 secs  
Retrieval task

In [0]:
plt.figure(figsize = (7, 3))
p1 = plt.plot(descriptor_history_baseline_trip.history['loss'], 'r')
p2 = plt.plot(descriptor_history_baseline_trip.history['val_loss'], 'g')
plt.title('De-noise Model Training Loss')
plt.ylabel('Triplet Loss')
plt.xlabel('Number of epochs')
plt.legend((p1[0], p2[0]), ('Training Loss', 'Test Loss'))

In [0]:
descriptor_model.save("descriptor.h5")
descriptor_model_trip.save("descriptor_trip.h5")

In [0]:
!zip -rq descriptors.zip ./out/custom